## Overview of how to set up mounted path and pre-install libraries/packages:

1. Establish a standard convention within your Organization and your shared workspace for `{project}/{dbr_version}/{Library}` &/or `{.ipython/profile/startup}` paths   

2. Install libraries and packages in the path identified above as `PYTHON_LIB_PATH_MOUNTED`   

3. Set up Cluster Configs:  
    - Create a `.ipython/profile_{pyenv}/startup/00_{pyenv}.py`file which executes the command to append `PYTHON_LIB_PATH_MOUNTED` with pre-installed python libraries to `sys.path()`.  

    - 3A [Recommended]: Set the Cluster Init Scripts with path to Volumes Init Script, which will create a symbolic `IPYTHON_PROFILE_PATH/startup` and `/root/ipython/profile_default/startup` paths.       
    - 3B [Alternative]: Create a Cluster Environment variable `IPYTHON_PROFILE_DIR` and set the  Cluster Init Scripts with path to Volumes Init Script, which will create a symbolic `IPYTHON_PROFILE_DIR/startup` and `/root/ipython/profile_default/startup` paths       


##### Post Installation of Library/Packages + Cluster Init Script / Environment Variable set up
With our Libs pre-installed + Cluster Init Script (via Volumes) and if required, Cluster Environment Variable set up, we will now make sure that the mounted path is `Read-Only`:   

4. Unmount + Remount Storage onto DBFS with Read-Only permissions     

5. Post Compute re-start/updates .... [TEST] Python_library loads!

### 1. Establish a standard convention within your Organization and your shared workspace for `{project}/{dbr_version}/{Library}` &/or `{.ipython/profile/}` paths

In [0]:
PROJECT_GROUP = "dais24_demo" 
PROJECT_DIR = "faster_lib_loads"
DBR_VERSION = "13.3LTS_ML" 

PATH_MOUNTED = f"/mnt/{PROJECT_GROUP}/" ## points to blobContainerName = "dais24" 
PYTHON_LIB_PATH_MOUNTED = f"/mnt/{PROJECT_GROUP}/{PROJECT_DIR}/{DBR_VERSION}/libs/python/" 

IPYTHON_PROFILE_PATH = f"/mnt/{PROJECT_GROUP}/{PROJECT_DIR}/{DBR_VERSION}/clusterEnv/.ipython/profile_pyenv/"

print('PATH_MOUNTED :', PATH_MOUNTED)
print('PYTHON_LIB_PATH_MOUNTED :', PYTHON_LIB_PATH_MOUNTED)
print('IPYTHON_PROFILE_PATH :', IPYTHON_PROFILE_PATH)


![Access Cloud Storage](./markdown_images/access_storage_container.png)

<!-- from IPython.display import Image
Image(filename="/Workspace/Users/{username@email.com}/Faster_Lib_Loads/markdown_images/access_storage_container.png", width=1600) -->

![generateSAStoken](./markdown_images/generateSAStoken.png)

<!-- %python
from IPython.display import Image
Image(filename="/Workspace/Users/{username@email.com}/Faster_Lib_Loads/markdown_images/generateSAStoken.png", width=1600) -->

In [0]:
secret_scope_name = "dais24_fasterlibloads" 
dbutils.secrets.list(f'{secret_scope_name}') 

In [0]:
storageAccountName = "hlsfieldexternal"
blobContainerName = "dais24" 
secret_scope_name = "dais24_fasterlibloads"
r_sasToken = dbutils.secrets.get(f'{secret_scope_name}','r_token')
rwr_sasToken = dbutils.secrets.get(f'{secret_scope_name}','rwr_token')

mountPoint = PATH_MOUNTED 

def mount_azblob(storageAccountName, blobContainerName, mountPoint, sasToken):

  # first unmount if already mounted 
  if any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.unmount(mountPoint)
    
  try:
    # mount to specified mountPoint
    dbutils.fs.mount(
      source = f"wasbs://{blobContainerName}@{storageAccountName}.blob.core.windows.net",
      mount_point = mountPoint,    
      extra_configs = {f"fs.azure.sas.{blobContainerName}.{storageAccountName}.blob.core.windows.net": sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

    dbutils.fs.refreshMounts()

In [0]:
mount_azblob(storageAccountName, blobContainerName, mountPoint, rwr_sasToken)

In [0]:
[p for p in dbutils.fs.mounts() if f"{blobContainerName}" in p.source]

In [0]:
print(PATH_MOUNTED)
display(dbutils.fs.ls(f'{PATH_MOUNTED}'))

In [0]:
PYTHON_LIB_PATH_MOUNTED

### 2. Install libraries and packages in the path identified above as `PYTHON_LIB_PATH_MOUNTED`. 

In [0]:
dbutils.fs.mkdirs(f"{PYTHON_LIB_PATH_MOUNTED}")

#### 2.1 Create os.environ target path variable for use with pip install
- NB: `/dbfs` prefix is required for correct path location 

In [0]:
import os
os.environ["PYTHON_LIB_PATH_MOUNTED"]=f'/dbfs{PYTHON_LIB_PATH_MOUNTED}'
os.environ["PYTHON_LIB_PATH_MOUNTED"]

In [0]:
%sh echo $PYTHON_LIB_PATH_MOUNTED

#### 2.2 Install libraries of interest to `$PYTHON_LIB_PATH_MOUNTED`

In [0]:
%sh pip install --upgrade pip

#### 2.2.1 Check that libraries aren't already installed on cluster by default 

In [0]:
import easydict

In [0]:
import torch_scatter

In [0]:
import torch_sparse

In [0]:
import torch_spline_conv

In [0]:
import torch_geometric

#### 2.2.2 Install libraries to `$PYTHON_LIB_PATH_MOUNTED`

In [0]:
# easydict torch-scatter torch-sparse torch-spline-conv torch_geometric

# Install commands took in total approx. ~45mins to install 

In [0]:
%sh pip install --upgrade easydict --target=$PYTHON_LIB_PATH_MOUNTED

In [0]:
%sh pip install --upgrade torch-scatter --target=$PYTHON_LIB_PATH_MOUNTED --verbose

In [0]:
%sh pip install --upgrade torch-sparse --target=$PYTHON_LIB_PATH_MOUNTED --verbose

In [0]:
%sh pip install --upgrade torch-spline-conv --target=$PYTHON_LIB_PATH_MOUNTED --verbose

In [0]:
%sh ls -l /usr/local | grep cuda

In [0]:
import torch;
torch.__version__ #'1.13.1+cu117'

In [0]:
%sh pip install torch_geometric --target=$PYTHON_LIB_PATH_MOUNTED --verbose

In [0]:
# dbutils.library.restartPython()

In [0]:
display(dbutils.fs.ls(f"{PYTHON_LIB_PATH_MOUNTED}")) 

#### 2.3 Append mounted path with pre-installed libraries to `sys.path`

In [0]:
import sys
sys.path.append(f"/dbfs{PYTHON_LIB_PATH_MOUNTED}") 

In [0]:
sys.path

#### 2.4 Quick test of appended mounted path with pre-installed libraries to sys.path

In [0]:
import os
import torch_geometric
path = os.path.abspath(torch_geometric.__file__)
path

In [0]:
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.nn import NNConv, BatchNorm, EdgePooling, TopKPooling, global_add_pool
from torch_geometric.utils import get_laplacian, to_dense_adj

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, Sigmoid, Tanh, Dropout, LeakyReLU
from torch.autograd import Variable
from torch.distributions import normal, kl

In [0]:
display(dbutils.fs.ls(f"{PYTHON_LIB_PATH_MOUNTED}"))

### 3. Set the Cluster Environment variable:

We will leverage the unique properties of the default `{##}_{filename}.py` files within `/root/.ipython/profile_default/startup/` path ([Ref](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#startup-files)).      

By symbolically linking our corresponding `IPYTHON_PROFILE_PATH` to the default `/root/.ipython/profile_default/startup/` path, our "bespoke" `.ipython/profile_pyenv/startup/00_pyenv.py` will execute our desired code for appending the `PYTHON_LIB_PATH_MOUNTED` with pre-installed python libraries to `sys.path()` during cluster initialization. 

To achieve this, we will: 

 - [3.1] Create a workspace `.ipython/profile/startup/00_pyenv.py` file which executes the command to append `PYTHON_LIB_PATH_MOUNTED` with pre-installed python libraries to `sys.path()`.  

 - [3.2] Copy the workspace `.ipython/profile/startup` files to our mounted external cloud storage `IPYTHON_PROFILE_PATH` with similar folder structure.  

 - [3.3] Use `init.sh` script in UC Volumes to create symbolic link between `{IPYTHON_PROFILE_PATH or IPYTHON_PROFILE_DIR}/startup` and `/root/ipython/profile_default/startup` paths.    

    This can be in the following form:   
     - [A] Direct association between `IPYTHON_PROFILE_PATH` to the default `/root/.ipython/profile_default/startup/` path.    

     - [B] Creating a Cluster Environment Variable : `IPYTHON_PROFILE_DIR` and associating `IPYTHON_PROFILE_DIR` to the default `/root/.ipython/profile_default/startup/` path -- this option means that you can update the cluster Advanced Environmental Variable without having to code-update the `init.sh` files.



#### 3.1 Create a workspace `.ipython/profile/startup/00_pyenv.py` file which executes the command to append `PYTHON_LIB_PATH_MOUNTED` with pre-installed python libraries to `sys.path()`.

In [0]:
%sh head /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/profile_pyenv/startup/00_pyenv.py

In [0]:
%sh ls -lah /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/profile_pyenv/startup/

In [0]:
%sh head /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/profile_pyenv/startup/README

#### 3.2 Copy the workspace `ipython/profile/startup` files to our mounted external cloud storage `IPYTHON_PROFILE_PATH` with similar folder structure.  

In [0]:
%sh cp -r /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/profile_pyenv /dbfs/mnt/dais24_demo/faster_lib_loads/13.3LTS_ML/clusterEnv/.ipython/

In [0]:
%fs head dbfs:/mnt/dais24_demo/faster_lib_loads/13.3LTS_ML/clusterEnv/.ipython/profile_pyenv/startup/00_pyenv.py

#### 3.3A Use init.sh scripts in UC Volumes to create symbolic link between `IPYTHON_PROFILE_PATH/startup` and `/root/ipython/profile_default/startup` paths. 

In [0]:
!head /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/ipython_profile_symlink2mnt_init.sh

In [0]:
!cp -r /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/ipython_profile_symlink2mnt_init.sh /Volumes/mmt_external/dais24/ext_vols/init_scripts/

In [0]:
print(dbutils.fs.head("/Volumes/mmt_external/dais24/ext_vols/init_scripts/ipython_profile_symlink2mnt_init.sh"))

In [0]:
from IPython.display import Image
Image(filename="/Workspace/Users/{username@email.com}/Faster_Lib_Loads/markdown_images/cluster_ipython_profile_symlink_initVol.png", width=1600)

#### 3.3B Use `init.sh` scripts in UC Volumes to create symbolic link between `IPYTHON_PROFILE_DIR/startup` and `/root/ipython/profile_default/startup paths`. 

In [0]:
!cat /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/ipython_profile_clusterEnvVar_init.sh

In [0]:
## copy .init script to mounted external UC Volumes path 
!cp -r /Workspace/Users/{username@email.com}/Faster_Lib_Loads/.ipython/ipython_profile_clusterEnvVar_init.sh /Volumes/mmt_external/dais24/ext_vols/init_scripts/

In [0]:
## check contents of coped .init script
print(dbutils.fs.head("/Volumes/mmt_external/dais24/ext_vols/init_scripts/ipython_profile_clusterEnvVar_init.sh"))

![cluster_ipython_profile_EnvVar_initVol](./markdown_images/cluster_ipython_profile_EnvVar_initVol.png)

<!-- %python
from IPython.display import Image
Image(filename="/Workspace/Users/{username@email.com}/Faster_Lib_Loads/markdown_images/cluster_ipython_profile_EnvVar_initVol.png", width=1600) -->

![cluster_ipython_profile_user_EnvVar](./markdown_images/cluster_ipython_profile_user_EnvVar.png)

<!-- %python
from IPython.display import Image
Image(filename="/Workspace/Users/{username@email.com}/Faster_Lib_Loads/markdown_images/cluster_ipython_profile_user_EnvVar.png", width=1600) -->

### 4 Unmount + Remount Storage onto DBFS with Read-Only permissions 
Now that the Python Lbs/packages are installed,   
We can unmount the (Azure blob) storage with `readNwrite` permissions    
And re-mount the storage path with `read-only` permissions    

In [0]:
mount_azblob(storageAccountName, blobContainerName, mountPoint, r_sasToken)

In [0]:
[p for p in dbutils.fs.mounts() if "dais24" in p.source]

In [0]:
%fs ls "dbfs:/mnt/dais24_demo/faster_lib_loads/13.3LTS_ML/libs/python/"

In [0]:
%fs mkdirs "dbfs:/mnt/dais24_demo/faster_lib_loads/13.3LTS_ML/libs/test"

### 5. TEST Cluster with new configs / RESTART Cluster!

In [0]:
import sys
sys.path

In [0]:
import os
import torch_geometric
path = os.path.abspath(torch_geometric.__file__)
path

In [0]:
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.nn import NNConv, BatchNorm, EdgePooling, TopKPooling, global_add_pool
from torch_geometric.utils import get_laplacian, to_dense_adj

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, Sigmoid, Tanh, Dropout, LeakyReLU
from torch.autograd import Variable
from torch.distributions import normal, kl

In [0]:
# import torch
# from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
Data(edge_index=[2, 4], x=[3, 1])


In [0]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())
data

In [0]:
data.validate(raise_on_error=True)

In [0]:
import networkx as nx

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = torch_geometric.data.Data(x=x, edge_index=edge_index)
g = torch_geometric.utils.to_networkx(data, to_undirected=True)
nx.draw(g)